In [8]:
%matplotlib inline
# %matplotlib notebook
import rasterio
from rasterio.coords import BoundingBox
from shapely.geometry import box
import geopandas as gpd
from rasterio.mask import mask
import os
import pandas as pd

In [9]:
url_imagen = 'D:\Memoria\Codigo\AraucoNursery\PROYECTOS\VIVERO\imagenes\imagen_mesones_rgb.tif'
shp = 'D:\Memoria\Codigo\AraucoNursery\PROYECTOS\VIVERO\puntos_plantas\plantas_mesones_2.shp'

In [10]:
shape = gpd.read_file(shp)

In [11]:
raster = rasterio.open(url_imagen)

In [12]:
offsetX = 0
offsetY = 0
widthX = 100
widthY = 100
step = 10
cuentas = pd.DataFrame(columns=['imagen', 'cant_arboles'])

In [13]:
def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [18]:
contador = 0
while (1):
        # finalizar si se a recorrido toda la imagen
    if(offsetY >= raster.height and offsetX >= raster.width):
        break
     #cuando llegue al borde en el eje x, debo bajar(sumar en eje y) y debo volver a empezar setear offsetX en 0 y widthX en 100
    elif(offsetX >= raster.width):
        offsetY = offsetY + step
        offsetX = 0
        widthX = 100 
   
    #entregamos los pixeles para obtener las coordenadas
    t = raster.xy(offsetX, offsetY)
    l = raster.xy(offsetX + widthX , offsetY)
    r = raster.xy(offsetX, offsetY+ widthY)
    b = raster.xy(offsetX + widthX , offsetY + widthY)  
    bbox = box(l[0], b[1], r[0], t[1])
    
    geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs=raster.crs)
    coords = getFeatures(geo)
    #print(coords)
    
    try:
        #transformacion de la imagen
#         out_img, out_transform = mask(raster, shapes=coords, crop=True)
#         out_meta = raster.meta.copy()
#         out_meta.update(
#         {"driver": "GTiff",
#               "height": 100,
#               "width": 100,
#               "transform": out_transform,
#               "crs": raster.crs})
#         with rasterio.open("train/salida"+str(contador)+".tif", "w", **out_meta) as dest:
#             dest.write(out_img)


#         print(type(shape))
#         print(type(geo))
    #interseccion ptos
        cantidad=geo.intersects(shape).value_counts()
        if (True in cantidad.index):
            numero = cantidad[True]
        else:
            numero = 0
        cuentas.loc[contador] = ["salida"+str(contador)+".tif",numero]
        if(contador%1000 == 0):
            print("salida"+str(contador)+".tif")

        
    except ValueError:
        print("train/salida"+str(contador)+".tif")
#         print(coords)
    contador = contador + 1
    offsetX = offsetX + step 
   

salida0.tif
salida1000.tif
salida2000.tif
salida3000.tif
salida4000.tif
salida5000.tif
salida6000.tif
salida7000.tif
salida8000.tif


KeyboardInterrupt: 

In [19]:
cuentas.to_csv('cuentas.csv',  encoding='utf-8')